# DIY Covid-19 Dashboard

The grapgh below displays data aquired from Public Health England (PHE), regarding new COVID-19 cases, hospital admissions and occupied beds with mechanical ventilators.
To display the data of intrest, select the corrsponding option in the selection box on the top left of the graph. To display multiple datasets on the graph, hold down the "Ctrl" and click on the relevent option fields.
In order to modifiy the scale of the Y-axis, select the approperiate option on the top right of the graph. To display a linear Y-axis select the "linear" option, and to display a logorithmic Y-axis select the "log" option.
To update the graph with the most up to date information aviable from PHE, click on the "Refersh Data" button.


In [114]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [115]:
with open("newcases.json", "rt") as FILE:
    info=json.load(FILE)

In [116]:
def parse_date(datestring):
    
    return pd.to_datetime(datestring, format="%Y-%m-%d")
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    infolist=info['data']

    day=[dictionary['date'] for dictionary in infolist ]
    day.sort()
    
    startdate=parse_date(day[0])
    enddate=parse_date(day[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    newcaseseriesdf=pd.DataFrame(index=index, columns=['Cases', "Admissions", "OccupiedMVBeds"])

    for entry in infolist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['Cases', "Admissions", "OccupiedMVBeds"]:
            if pd.isna(newcaseseriesdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                newcaseseriesdf.loc[date, column]=value
    return newcaseseriesdf

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
newcaseseriesdf=wrangle_data(info) # df is the dataframe for plotting

In [117]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 

def access_api():
    new_covid_cases = {
    'date' : 'date',
    'Cases' : 'newCasesByPublishDate',
    'Admissions' : 'newAdmissions',
    'OccupiedMVBeds' : 'covidOccupiedMVBeds' 
    }

    within_england = ['areaType=nation', 'areaName=England']

    api = Cov19API(filters=within_england, structure=new_covid_cases)
    newcases=api.get_json()

    with open("newcases.json", "wt") as FILE:
        json.dump(newcases, FILE)
        
    print("I'm downloading data from the API...")
    print("...all done.")
    return newcases # return data read from the API


# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    #global newcaseseriesdf
    newcaseseriesdf=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True


apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

In [118]:



cases_cols=wdg.SelectMultiple(
    options=['Cases', 'Admissions', 'OccupiedMVBeds'], # options available
    value=['Cases'], # initial value
    rows=3, # rows of the selection box
    description='Select',
    disabled=False)




case_scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
con=wdg.HBox([cases_cols, case_scale])


def case_graph(casecolumns, casescale):
    # our callback function.
    if casescale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(casecolumns)
    if ncols>0:
         newcaseseriesdf[list(casecolumns)].plot(logy=logscale)
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=case_scale.value
    if current==case_scale.options[0]:
        other=case_scale.options[1]
    else:
        other=case_scale.options[0]
    case_scale.value=other # forces the redraw
    case_scale.value=current # now we can change it back
    
# keep calling age_graph(graphcolumns=value_of_agecols); capture output in variable output    
outputs=wdg.interactive_output(case_graph, {'casecolumns': cases_cols, "casescale": case_scale})

display(con, outputs)



Output()